In [ ]:
import pandas as pd

df=pd.read_csv('metadata.csv')

In [ ]:
df.head()

,Unnamed: 0,image,class,format,mode,shape
0,0,Cancer (1).jpg,tumor,JPEG,RGB,"(512, 512, 3)"
1,1,Cancer (1).png,tumor,PNG,L,"(300, 240)"
2,2,Cancer (1).tif,tumor,TIFF,RGB,"(256, 256, 3)"
3,3,Cancer (10).jpg,tumor,JPEG,RGB,"(512, 512, 3)"
4,4,Cancer (10).tif,tumor,TIFF,RGB,"(256, 256, 3)"


In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size=0.2, random_state=42)
print(len(train),len(val))


3680 920


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    validation_split=0.2 # Set validation split here
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory='./', # Replace with your image directory
    x_col="image", # Replace with your image column name
    y_col="class", # Replace with your label column name
    subset="training",
    batch_size=32,
    seed=42,
    target_size=(224, 224),  # Adjust as needed
    class_mode='categorical'  # Adjust as needed
)

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=val,
    directory='./', # Replace with your image directory
    x_col="image",  # Replace with your image column name
    y_col="class", # Replace with your label column name
    subset="validation",
    batch_size=32,
    seed=42,
    target_size=(224, 224),  # Adjust as needed
    class_mode='categorical' # Adjust as needed
)


Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 3680 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 920 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  # Added dropout for regularization
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),  # Added dropout for regularization
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use binary_crossentropy for binary classification
              metrics=['accuracy'])

model.summary()
